In [1]:
!pip install evaluate rouge_score --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
import spacy
import evaluate

In [ ]:
cnn_dt= load_dataset('cnn_dailymail', '3.0.0')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
cnn_dt

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
cnn_dt['train'][0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [ ]:
# To ccheck empty and null
cnn_dt['train'].filter(lambda example: len(example['article'])==0)

Filter:   0%|          | 0/287113 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 0
})

In [ ]:
cnn_dt['train'].filter(lambda example: len(example['highlights'])==0)

Filter:   0%|          | 0/287113 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 0
})

In [ ]:
cnn_dt['train'].filter(lambda example: len(example['id'])==0)


Filter:   0%|          | 0/287113 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 0
})

In [ ]:
# Summaraize first

B_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
B_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

f_tokenized = B_tokenizer(cnn_dt['train'][0]['article'], return_tensors='pt', truncation=True)
f_modeled = B_model.generate(**f_tokenized)

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
summary_ids = B_model.generate(f_tokenized['input_ids'], num_beams=4,max_length=100, early_stopping=True)

In [ ]:
decoded_summary = B_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(decoded_summary)
cnn_dt['train'][0]['highlights']

Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe's earnings from the first five Potter films have been held in a trust fund. Details of how he'll mark his landmark birthday are under wraps.


"Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund ."

In [ ]:
nlp=spacy.load('en_core_web_sm')
doc1 = nlp(decoded_summary)
doc2 = nlp(cnn_dt['train'][0]['highlights'])
doc1.similarity(doc2)

/tmp/ipython-input-1825773528.py:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  doc1.similarity(doc2)


0.8422773480415344

In [ ]:
# evaluation more coverying and technical

rouge = evaluate.load('rouge')
predictions = [decoded_summary]
references = [cnn_dt['train'][0]['highlights']]

rouge.compute(predictions=predictions, references=references)


{'rouge1': np.float64(0.6136363636363635),
 'rouge2': np.float64(0.372093023255814),
 'rougeL': np.float64(0.5454545454545454),
 'rougeLsum': np.float64(0.5681818181818182)}